# CLs limit with MC toys

In [1]:
%jsroot

Open the `ROOT` file

In [2]:
TFile* f = TFile::Open("../workspaces/model.root") ;


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



Retrieve the workspace

In [3]:
RooWorkspace* w = (RooWorkspace*) f->Get("w") ;
w->Print() ;


RooWorkspace(w) w contents

variables
---------
(B_nom,B_systerr,Nobs_SR,S,alphaB_nom,alpha_B,mu)

p.d.f.s
-------
RooProdPdf::model[ model_SR * model_alphaB ] = 0.0511153
RooPoisson::model_SR[ x=Nobs_SR mean=Nexp_SR ] = 0.0511153
RooGaussian::model_alphaB[ x=alphaB_nom mean=alpha_B sigma=1 ] = 1

functions
--------
RooFormulaVar::B[ actualVars=(B_nom,alpha_B,B_systerr) formula="B_nom+alpha_B*B_systerr" ] = 20
RooFormulaVar::Nexp_SR[ actualVars=(mu,S,B) formula="mu*S+B" ] = 30

datasets
--------
RooDataSet::observed_data(Nobs_SR)

parameter snapshots
-------------------
ModelConfig__snapshot = (mu=1)

named sets
----------
ModelConfig_GlobalObservables:(alphaB_nom)
ModelConfig_NuisParams:(alpha_B)
ModelConfig_Observables:(Nobs_SR)
ModelConfig_POI:(mu)
ModelConfig__snapshot:(mu)

generic objects
---------------
RooStats::ModelConfig::ModelConfig



### Retrieve the ModelConfig for the S+B hypothesis

Retrieve the ModelConfig and the observed data. Together these uniquely define the statistical problem

In [4]:
RooAbsData* data = w->data("observed_data") ;
RooStats::ModelConfig* sbModel = (RooStats::ModelConfig*) w->obj("ModelConfig") ;  

## Construct a ModelConfig for the B-only hypothesis

For a CLS-style limit calculation (hypothesis test inversion) we need an explicit specification of the background-only hypothesis == another RooStats::ModelConfig that describe the B-only scenario  

In [5]:
RooStats::ModelConfig* bModel = (RooStats::ModelConfig*) sbModel->Clone("BonlyModel") ;

Here we take a little shortcut from universality by assuming that the POI=0 scenario corresponds to the background-only scenario

Set value POI parameter to zero

In [6]:
RooRealVar* poi = (RooRealVar*) bModel->GetParametersOfInterest()->first();
poi->setVal(0) ;

Configure bModel to encode current poi=0 scenario as its hypothesis 

In [7]:
bModel->SetSnapshot( *poi  );

*NB: To make CLS-style hypothesis calculation macros truly universal workspace files should contain both ModelConfigs upfront*

### Construct an hypothesis p-value calculator
i.e the calculation of p(sbModel) and p(bModel) for the observed data

Instantiate hypothesis testing calculator assuming asymptotic distributions of the profile likelihood ratio (`PLR`) test statistic. This calculator is much more time consuming than the asymptotic calculator but is also valid in the low statistics regime.

In [8]:
RooStats::FrequentistCalculator  freqCalc(*data, *bModel, *sbModel);

The frequentist calculator is more general than the asymptotic calculator: it can calculate distributions for _any_ test statistic. So here we have to tell it that we want the profile likelihood ratio test statistic

In [9]:
RooStats::ProfileLikelihoodTestStat* plr = new RooStats::ProfileLikelihoodTestStat(*sbModel->GetPdf());

Configure calculator for a limit (=one-sided interval)

In [10]:
plr->SetOneSided(true); 

Specifically we have to tell the Toy MC sampler part of the calculator what the relevant test statistic is

In [11]:
RooStats::ToyMCSampler* toymcs = (RooStats::ToyMCSampler*) freqCalc.GetTestStatSampler();
toymcs->SetTestStatistic(plr);

If we use the frequentist calculator for counting experiments (instead of models of distributions) we should instruct the sampler to generate one event for each toy. ( This is the case because we model counting experiments in RooFit as a single observation in distribution of event counts. )

In [12]:
if (!sbModel->GetPdf()->canBeExtended()) {
    toymcs->SetNEventsPerToy(1);
}

Sample 1000 toys for SB and B hypothesis respectively to model their distributions (Here you can trade speed vs accuracy)

In [13]:
freqCalc.SetToys(1000,1000) ;

### Construct an hypothesis test inverter

i.e. a tool that can calculate the POI value for which (in this case) CLS==$p(\mathrm{sbModel})/(1-p(\mathrm{Model}))$ takes a certain value. This inversion requires a scan over possible values of $\mu$.

In [14]:
RooStats::HypoTestInverter inverter(freqCalc);

[#1] INFO:InputArguments -- HypoTestInverter ---- Input models: 
		 using as S+B (null) model     : ModelConfig
		 using as B (alternate) model  : BonlyModel



Statistical configuration of hypothesis test inverter

In [15]:
inverter.SetConfidenceLevel(0.90);
inverter.UseCLs(true); 

Technical configuration of hypothesis test inverter

In [16]:
inverter.SetVerbose(false);
inverter.SetFixedScan(30,0.0,6.0); // set number of points , xmin and xmax 

Perform calculation of limit

In [17]:
RooStats::HypoTestInverterResult* result =  inverter.GetInterval();

[#1] INFO:Eval -- HypoTestInverter::GetInterval - run a fixed scan
[#1] INFO:ObjectHandling -- RooWorkspace::saveSnaphot(w) replacing previous snapshot with name ModelConfig__snapshot

=== Using the following for ModelConfig ===
Observables:             RooArgSet:: = (Nobs_SR)
Parameters of Interest:  RooArgSet:: = (mu)
Nuisance Parameters:     RooArgSet:: = (alpha_B)
Global Observables:      RooArgSet:: = (alphaB_nom)
PDF:                     RooProdPdf::model[ model_SR * model_alphaB ] = 0.0445876
Snapshot:                
  1) 0x7fea8158e3e0 RooRealVar:: mu = 0  L(-1 - 10)  "mu"


=== Using the following for BonlyModel ===
Observables:             RooArgSet:: = (Nobs_SR)
Parameters of Interest:  RooArgSet:: = (mu)
Nuisance Parameters:     RooArgSet:: = (alpha_B)
Global Observables:      RooArgSet:: = (alphaB_nom)
PDF:                     RooProdPdf::model[ model_SR * model_alphaB ] = 0.0445876
Snapshot:                
  1) 0x7fea8158e3e0 RooRealVar:: mu = 0  L(-1 - 10)  "mu"

[#0] 

[#0] PROGRESS:Generation -- generated toys: 500 / 1000
[#1] INFO:ObjectHandling -- RooWorkspace::saveSnaphot(w) replacing previous snapshot with name ModelConfig__snapshot

=== Using the following for ModelConfig ===
Observables:             RooArgSet:: = (Nobs_SR)
Parameters of Interest:  RooArgSet:: = (mu)
Nuisance Parameters:     RooArgSet:: = (alpha_B)
Global Observables:      RooArgSet:: = (alphaB_nom)
PDF:                     RooProdPdf::model[ model_SR * model_alphaB ] = 0.031657
Snapshot:                
  1) 0x7fea816eac20 RooRealVar:: mu = 1.24138  L(-1 - 10)  "mu"


=== Using the following for BonlyModel ===
Observables:             RooArgSet:: = (Nobs_SR)
Parameters of Interest:  RooArgSet:: = (mu)
Nuisance Parameters:     RooArgSet:: = (alpha_B)
Global Observables:      RooArgSet:: = (alphaB_nom)
PDF:                     RooProdPdf::model[ model_SR * model_alphaB ] = 0.031657
Snapshot:                
  1) 0x7fea816eac20 RooRealVar:: mu = 0  L(-1 - 10)  "mu"

[#0] PROGRESS

[#0] PROGRESS:Generation -- generated toys: 500 / 1000
[#1] INFO:ObjectHandling -- RooWorkspace::saveSnaphot(w) replacing previous snapshot with name ModelConfig__snapshot

=== Using the following for ModelConfig ===
Observables:             RooArgSet:: = (Nobs_SR)
Parameters of Interest:  RooArgSet:: = (mu)
Nuisance Parameters:     RooArgSet:: = (alpha_B)
Global Observables:      RooArgSet:: = (alphaB_nom)
PDF:                     RooProdPdf::model[ model_SR * model_alphaB ] = 0.000426201
Snapshot:                
  1) 0x7fea817758a0 RooRealVar:: mu = 2.48276  L(-1 - 10)  "mu"


=== Using the following for BonlyModel ===
Observables:             RooArgSet:: = (Nobs_SR)
Parameters of Interest:  RooArgSet:: = (mu)
Nuisance Parameters:     RooArgSet:: = (alpha_B)
Global Observables:      RooArgSet:: = (alphaB_nom)
PDF:                     RooProdPdf::model[ model_SR * model_alphaB ] = 0.000426201
Snapshot:                
  1) 0x7fea817758a0 RooRealVar:: mu = 0  L(-1 - 10)  "mu"

[#0] PR

[#0] PROGRESS:Generation -- generated toys: 500 / 1000
[#1] INFO:ObjectHandling -- RooWorkspace::saveSnaphot(w) replacing previous snapshot with name ModelConfig__snapshot

=== Using the following for ModelConfig ===
Observables:             RooArgSet:: = (Nobs_SR)
Parameters of Interest:  RooArgSet:: = (mu)
Nuisance Parameters:     RooArgSet:: = (alpha_B)
Global Observables:      RooArgSet:: = (alphaB_nom)
PDF:                     RooProdPdf::model[ model_SR * model_alphaB ] = 7.80713e-07
Snapshot:                
  1) 0x7fea8199b230 RooRealVar:: mu = 3.72414  L(-1 - 10)  "mu"


=== Using the following for BonlyModel ===
Observables:             RooArgSet:: = (Nobs_SR)
Parameters of Interest:  RooArgSet:: = (mu)
Nuisance Parameters:     RooArgSet:: = (alpha_B)
Global Observables:      RooArgSet:: = (alphaB_nom)
PDF:                     RooProdPdf::model[ model_SR * model_alphaB ] = 7.80713e-07
Snapshot:                
  1) 0x7fea8199b230 RooRealVar:: mu = 0  L(-1 - 10)  "mu"

[#0] PR

[#0] PROGRESS:Generation -- generated toys: 500 / 1000
[#1] INFO:ObjectHandling -- RooWorkspace::saveSnaphot(w) replacing previous snapshot with name ModelConfig__snapshot

=== Using the following for ModelConfig ===
Observables:             RooArgSet:: = (Nobs_SR)
Parameters of Interest:  RooArgSet:: = (mu)
Nuisance Parameters:     RooArgSet:: = (alpha_B)
Global Observables:      RooArgSet:: = (alphaB_nom)
PDF:                     RooProdPdf::model[ model_SR * model_alphaB ] = 4.28876e-10
Snapshot:                
  1) 0x7fea8199b230 RooRealVar:: mu = 4.96552  L(-1 - 10)  "mu"


=== Using the following for BonlyModel ===
Observables:             RooArgSet:: = (Nobs_SR)
Parameters of Interest:  RooArgSet:: = (mu)
Nuisance Parameters:     RooArgSet:: = (alpha_B)
Global Observables:      RooArgSet:: = (alphaB_nom)
PDF:                     RooProdPdf::model[ model_SR * model_alphaB ] = 4.28876e-10
Snapshot:                
  1) 0x7fea8199b230 RooRealVar:: mu = 0  L(-1 - 10)  "mu"

[#0] PR

[#0] PROGRESS:Generation -- generated toys: 500 / 1000


Print observed limit

In [18]:
cout << 100*inverter.ConfidenceLevel() << "%  upper limit : " << result->UpperLimit() << endl;

90%  upper limit : 1.42225


compute expected limit

In [19]:
std::cout << "Expected upper limits, using the B (alternate) model : " << std::endl;
std::cout << " expected limit (median) " << result->GetExpectedUpperLimit(0) << std::endl;
std::cout << " expected limit (-1 sig) " << result->GetExpectedUpperLimit(-1) << std::endl;
std::cout << " expected limit (+1 sig) " << result->GetExpectedUpperLimit(1) << std::endl;
std::cout << " expected limit (-2 sig) " << result->GetExpectedUpperLimit(-2) << std::endl;
std::cout << " expected limit (+2 sig) " << result->GetExpectedUpperLimit(2) << std::endl;

Expected upper limits, using the B (alternate) model : 
 expected limit (median) 1.08172
 expected limit (-1 sig) 0.762889
 expected limit (+1 sig) 1.59385
 expected limit (-2 sig) 0.547087
 expected limit (+2 sig) 2.19304


Use the visualization tool of the PLC to show how the interval was calculated

In [20]:
TCanvas* c1 = new TCanvas();
RooStats::HypoTestInverterPlot* plot = new RooStats::HypoTestInverterPlot("HTI_Result_Plot","HypoTest Scan Result",result);
plot->Draw("CLb 2CL");  // plot also CLb and CLs+b 
c1->Draw()